In [1]:
# Seed value
# Apparently you may use different seed values at each stage
seed_value= 48

# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set the `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
# tf.random.set_seed(seed_value)
# for later versions: 
tf.compat.v1.set_random_seed(seed_value)

# 5. Configure a new global `tensorflow` session
from keras import backend as K
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

Using TensorFlow backend.


In [0]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt

In [0]:
from zipfile import ZipFile

with ZipFile('/content/drive/My Drive/GALA.zip', 'r') as zip:
  zip.extractall()

In [4]:
train=pd.read_csv('/content/dataset/train.csv')
test=pd.read_csv('/content/dataset/test.csv')
train.head()

,Image,Class
0,image7042.jpg,Food
1,image3327.jpg,misc
2,image10335.jpg,Attire
3,image8019.jpg,Food
4,image2128.jpg,Attire


In [0]:
Class_map={'Food':0,'Attire':1,'Decorationandsignage':2,'misc':3}
inverse_map={0:'Food',1:'Attire',2:'Decorationandsignage',3:'misc'}
train['Class']=train['Class'].map(Class_map)

In [6]:
train['Class']

0       0
1       3
2       1
3       0
4       1
       ..
5978    0
5979    1
5980    0
5981    0
5982    1
Name: Class, Length: 5983, dtype: int64

In [0]:
h, w = 164, 164

In [8]:
from tensorflow.keras.applications.resnet import preprocess_input

train_img=[]
train_label=[]
j=0
path='/content/dataset/Train Images'
for i in tqdm(train['Image']):
    final_path=os.path.join(path,i)
    img=cv2.imread(final_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img=cv2.resize(img,(h,w))
    img=np.array(img, dtype=np.float32)
    img /= 255.
    img -= 0.5
    img *= 2.
    train_img.append(img)
    train_label.append(train['Class'][j])
    j=j+1

100%|██████████| 5983/5983 [00:03<00:00, 1877.96it/s]


In [9]:
test_img=[]
path='/content/dataset/Test Images'
for i in tqdm(test['Image']):
    final_path=os.path.join(path,i)
    img=cv2.imread(final_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img=cv2.resize(img,(h,w))
    img=np.array(img, dtype=np.float32)
    img /= 255.
    img -= 0.5
    img *= 2.
    test_img.append(img)

100%|██████████| 3219/3219 [00:01<00:00, 2075.17it/s]


In [10]:
train_img=np.array(train_img)
test_img=np.array(test_img)
train_label=np.array(train_label)
print(train_img.shape)
print(test_img.shape)
print(train_label.shape)

(5983, 164, 164, 3)
(3219, 164, 164, 3)
(5983,)


In [11]:
from tensorflow.keras.applications.vgg16 import VGG16
base_model = VGG16(include_top=False, weights='imagenet', input_shape=(h, w, 3), pooling='avg')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
train_features = base_model.predict(train_img)
test_features = base_model.predict(test_img)

In [0]:
import xgboost as xgb
model = xgb.XGBClassifier()

In [14]:
model.fit(train_features, train_label)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [16]:
label = model.predict(test_features)
class_label = [inverse_map[x] for x in label]
print(class_label[:3])
submission = pd.DataFrame({ 'Image': test.Image, 'Class': class_label })
submission.head(10)
submission.to_csv('submission.csv', index=False)

['Food', 'Attire', 'Food']
